In [1]:
import gtsam
import numpy as np

In [2]:
def compute_matrix_spec(array):
    spec = ''
    
    for row in range(array.shape[0]):
        if row!=0:
            spec+=' '    
        for col in range(array.shape[1]):
            if col!=0:
                spec+='/'
            spec+=str(array[row,col])
    
    return spec

In [142]:
class_sym = gtsam.Symbol('c',0)
exist_sym = gtsam.symbol('e',0)
det_sym = gtsam.symbol('d',0)

print(class_sym.key())

7133701809754865664


# FROM THE TOP - COMBINED EXISTENCE AND CLASSIFICATION

In [26]:
# Form semantic sensor model and detection probability model
classes = ['bicycle',
           'car',
           'pedestrian',
           'void_ignore']

labels = ['barrier',
          'bicycle',
          'car',
          'pedestrian',
          'traffic_cone']

exists = ['does not exist',
          'exists']

false_pos_rate = .45
missed_det_rate = .2

class_sym = gtsam.symbol('c',1)
label_sym = gtsam.symbol('l',1)
exist_sym = gtsam.symbol('e',1)
det_sym = gtsam.symbol('d',1)

# Form existence and label probabilities
p_l_c_matrix = np.array([[0, 1., 0., 0, 0],[0, 0, 1., 0, 0],[0, 0., 0, 1., 0],[.5, 0, 0., 0, .5]])
p_l_c_str = compute_matrix_spec(p_l_c_matrix)

p_e_d_matrix = np.array([[false_pos_rate, 1 - false_pos_rate],[1 - missed_det_rate, missed_det_rate]])
p_e_d_str = compute_matrix_spec(p_e_d_matrix)
# Create conditional probability matrices

p_label_class = gtsam.DiscreteConditional((label_sym,len(labels)),[(class_sym,len(classes))],p_l_c_str)
p_exists_detected = gtsam.DiscreteConditional((exist_sym,2),[(det_sym,2)],p_e_d_str)

print(p_exists_detected)

Discrete Conditional
 P( e1 | d1 ):
 Choice(e1) 
 0 Choice(d1) 
 0 0 Leaf 0.45
 0 1 Leaf  0.8
 1 Choice(d1) 
 1 0 Leaf 0.55
 1 1 Leaf  0.2




In [134]:
# Simulate initialization
# labels = ['barrier', 'bicycle', 'car', 'pedestrian', 'traffic_cone']
det_idx = 0 # 0 = detected, 1 = nothing
label_idx = 1
factor = p_label_class.likelihood(label_idx)
class_dist = gtsam.DiscreteDistribution(factor)

# print(class_dist)
print(classes[class_dist.argmax()])
print(class_dist(class_dist.argmax()))

existence = gtsam.DiscreteDistribution(p_exists_detected.likelihood(0))
print(exists[existence.argmax()])
print(existence(existence.argmax()))

bicycle
1.0
exists
0.64


In [136]:
# Simulate update
label_idx = -1 # missed detection

# IF detection available
if label_idx>=0:
    print('detected %s' % labels[label_idx])
    factor = p_label_class.likelihood(label_idx)
    class_dist = gtsam.DiscreteDistribution(factor*class_dist)
    existence = gtsam.DiscreteDistribution(p_exists_detected.likelihood(0)*existence)
else:
    print('missed detection')
    existence = gtsam.DiscreteDistribution(p_exists_detected.likelihood(1)*existence)

print(classes[class_dist.argmax()])
print(class_dist(class_dist.argmax()))

print(exists[existence.argmax()])
print(existence(existence.argmax()))
print(existence.argmax())

missed detection
bicycle
1.0
does not exist
0.8096654275092937
0


# Test/dev - track confidence

In [2]:
# sensor model p(X|Z)
classes = ['false_detection',
           'void_ignore',
           'bicycle',
           'bus',
           'car',
           'motorcycle',
           'pedestrian',
           'trailer',
           'truck']
class_idx = 123

# detection
detections = ['missed_detection',
              'barrier',
              'bicycle',
              'bus',
              'car',
              'motorcycle',
              'pedestrian',
              'traffic_cone',
              'trailer',
              'truck']
det_idx = 456


In [91]:
test_classes = ['doesnt_exist',
               'exists']
detected_classes = ['detected',
                    'not_detected']

exists_idx = 10
det_idx = 20

test_cond_matrix = np.array([[.1, .9],[.8, .2]])
print(test_cond_matrix)

exist_spec_str = compute_matrix_spec(test_cond_matrix)
print(exist_spec_str)

exists_conf = .55
track_prior = gtsam.DiscreteDistribution((exists_idx,2),[trk_conf, 1-trk_conf])
print(track_prior)

# Sensor model
p_detection_existence = gtsam.DiscreteConditional((det_idx,2),[(exists_idx,2)],exist_spec_str)
print(p_detection_existence)


# Detection indices
det_idx = 0
missed_idx = 1 

det_factor = p_detection_existence.likelihood(det_idx)
missed_factor = p_detection_existence.likelihood(missed_idx)
print(det_factor)
print(missed_factor)


[[0.1 0.9]
 [0.8 0.2]]
0.1/0.9 0.8/0.2
Discrete Prior
 P( 10 ):
 Choice(10) 
 0 Leaf 0.55
 1 Leaf 0.45


Discrete Conditional
 P( 20 | 10 ):
 Choice(20) 
 0 Choice(10) 
 0 0 Leaf  0.1
 0 1 Leaf  0.8
 1 Choice(10) 
 1 0 Leaf  0.9
 1 1 Leaf  0.2


DecisionTreeFactor
 f[ (10,2), ]
 Choice(10) 
 0 Leaf  0.1
 1 Leaf  0.8

DecisionTreeFactor
 f[ (10,2), ]
 Choice(10) 
 0 Leaf  0.9
 1 Leaf  0.2



In [21]:
idx = 1 # 0 for det, 1 for missed

factor = p_detection_existence.likelihood(idx)

track_prior = gtsam.DiscreteDistribution(factor*track_prior)

print(track_prior)
print(test_classes[track_prior.argmax()])
print(track_prior(track_prior.argmax()))


NameError: name 'p_detection_existence' is not defined

# Test/Dev - class probability

In [56]:
test_classes = ['void_ignore',
                'person',
                'furniture']
test_det_labels = ['no_leg',
                   'leg']

cat_idx = 10
det_idx = 20

test_cond_matrix = np.array([[.9, .1],[.2, .8],[.4, .6]])
print(test_cond_matrix)

class_spec_str = compute_matrix_spec(test_cond_matrix)
print(class_spec_str)




[[0.9 0.1]
 [0.2 0.8]
 [0.4 0.6]]
0.9/0.1 0.2/0.8 0.4/0.6
0.8/0.2


In [60]:
# Compute prior
n_test_cats = len(test_classes)
n_test_dets = len(test_detections)

category_prior = gtsam.DiscreteDistribution((cat_idx,n_test_cats),[1/n_test_cats]*n_test_cats)
print(category_prior)


Discrete Prior
 P( 10 ):
 Leaf 0.33333333


Discrete Prior
 P( 30 ):
 Choice(30) 
 0 Leaf 0.55
 1 Leaf 0.45




In [64]:
# Sensor model
p_Z_X = gtsam.DiscreteConditional((det_idx,n_test_dets),[(cat_idx,n_test_cats)],class_spec_str)
print(p_Z_X)

p_detected_exists = gtsam.DiscreteConditional((det_idx,2),[(cat_idx,1)],exist_spec_str)
print(p_detected_exists)


# Detection indices
no_leg_det_idx = 0
leg_det_idx = 1 

Discrete Conditional
 P( 20 | 10 ):
 Choice(20) 
 0 Choice(10) 
 0 0 Leaf  0.9
 0 1 Leaf  0.2
 0 2 Leaf  0.4
 1 Choice(10) 
 1 0 Leaf  0.1
 1 1 Leaf  0.8
 1 2 Leaf  0.6


Discrete Conditional
 P( 20 | 10 ):
 Choice(20) 
 0 Leaf  0.8
 1 Leaf  0.2




In [33]:
print(dir(p_Z_X))

no_leg_detected_factor = p_Z_X.likelihood(no_leg_det_idx)
print(no_leg_detected_factor)

leg_detected_factor = p_Z_X.likelihood(leg_det_idx)
print(leg_detected_factor)


['__call__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__truediv__', '_repr_html_', '_repr_markdown_', 'cardinality', 'choose', 'dot', 'empty', 'enumerate', 'equals', 'error', 'evaluate', 'firstFrontalKey', 'keys', 'likelihood', 'logNormalizationConstant', 'logProbability', 'marginal', 'max', 'nrFrontals', 'nrParents', 'print', 'printKeys', 'printSignature', 'sample', 'sampleInPlace', 'size', 'sum']
Discrete Conditional
 P( 20 | 10 ):
 Choice(20) 
 0 Choice(10) 
 0 0 Leaf  0.9
 0 1 Leaf  0.2
 0 2 Leaf  0.4
 1 Choice(10) 
 1 0 Leaf  0.1
 1 1 Leaf  0.8
 1 2 Leaf  0.6


DecisionTreeFactor
 f[ (10,3), ]
 Choice(10) 
 0 Leaf  0.9
 1 Leaf  0.2
 2 Leaf  0.4

DecisionTreeFactor
 f[ (10,3), ]
 Choice(10) 
 0 Leaf

TypeError: __call__(): incompatible function arguments. The following argument types are supported:
    1. (self: gtsam.gtsam.DecisionTreeFactor, arg0: gtsam::DiscreteValues) -> float

Invoked with: DecisionTreeFactor
 f[ (10,3), ]
 Choice(10) 
 0 Leaf  0.1
 1 Leaf  0.8
 2 Leaf  0.6
, 10

In [13]:
norm_posterior = gtsam.DiscreteDistribution(leg_detected_factor)
print(norm_posterior)

Discrete Prior
 P( 10 ):
 Choice(10) 
 0 Leaf 0.066666667
 1 Leaf 0.53333333
 2 Leaf  0.4




In [15]:
meas_idx = 1 # 0 for no leg, 1 for leg detected
unnorm_posterior = p_Z_X.likelihood(meas_idx)*norm_posterior
norm_posterior = gtsam.DiscreteDistribution(unnorm_posterior)
print(norm_posterior)
print()
print(test_classes[norm_posterior.argmax()])
print(norm_posterior(norm_posterior.argmax()))

Discrete Prior
 P( 10 ):
 Choice(10) 
 0 Leaf 0.0013717421
 1 Leaf 0.70233196
 2 Leaf 0.2962963



person
0.7023319615912209


In [142]:
help(gtsam.DiscreteDistribution)

Help on class DiscreteDistribution in module gtsam.gtsam:

class DiscreteDistribution(DiscreteConditional)
 |  Method resolution order:
 |      DiscreteDistribution
 |      DiscreteConditional
 |      DecisionTreeFactor
 |      DiscreteFactor
 |      Factor
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __call__(...)
 |      __call__(self: gtsam.gtsam.DiscreteDistribution, arg0: int) -> float
 |  
 |  __init__(...)
 |      __init__(*args, **kwargs)
 |      Overloaded function.
 |      
 |      1. __init__(self: gtsam.gtsam.DiscreteDistribution) -> None
 |      
 |      2. __init__(self: gtsam.gtsam.DiscreteDistribution, f: gtsam.gtsam.DecisionTreeFactor) -> None
 |      
 |      3. __init__(self: gtsam.gtsam.DiscreteDistribution, key: Tuple[int, int], spec: str) -> None
 |      
 |      4. __init__(self: gtsam.gtsam.DiscreteDistribution, key: Tuple[int, int], spec: List[float]) -> None
 |  
 |  __repr__(...)
 |      __repr__(se

# The real thing

In [18]:
n_categories = len(classes)
category_prior = gtsam.DiscreteDistribution((0,n_categories),[1/n_categories]*n_categories)
print(category_prior)

Discrete Prior
 P( 0 ):
 Leaf 0.066666667




In [23]:
# there are n_category rows, with n_detections columns
# each row represents the likelihood that the category will yield that detection class, and each row sums to 1
p_Z_X = gtsam.DiscreteConditional((1,n_detections),[(0,n_categories)],
                                 "1/1/1/1/1/1/1/1/1/1/1/1 2/8/0/0/0/0/0/0/0/0/0/0 ")
help(gtsam.DiscreteConditional)

Help on class DiscreteConditional in module gtsam.gtsam:

class DiscreteConditional(DecisionTreeFactor)
 |  Method resolution order:
 |      DiscreteConditional
 |      DecisionTreeFactor
 |      DiscreteFactor
 |      Factor
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(...)
 |      __init__(*args, **kwargs)
 |      Overloaded function.
 |      
 |      1. __init__(self: gtsam.gtsam.DiscreteConditional) -> None
 |      
 |      2. __init__(self: gtsam.gtsam.DiscreteConditional, nFrontals: int, f: gtsam.gtsam.DecisionTreeFactor) -> None
 |      
 |      3. __init__(self: gtsam.gtsam.DiscreteConditional, key: Tuple[int, int], spec: str) -> None
 |      
 |      4. __init__(self: gtsam.gtsam.DiscreteConditional, key: Tuple[int, int], parents: gtsam.gtsam.DiscreteKeys, spec: str) -> None
 |      
 |      5. __init__(self: gtsam.gtsam.DiscreteConditional, key: Tuple[int, int], parents: List[Tuple[int, int]], spec: str) ->